In [25]:
import pandas as pd
from bs4 import BeautifulSoup

In [26]:
def dataset_list(dataset):
    
    dataset = str(dataset.find('p').text)
    dataset = dataset.split('\n')
    dataset = list(filter(None, dataset))

    dataset_arranged = []
    
    for i in range(0,len(dataset),3):
        temp = []
        temp.append(dataset[i])
        temp.append(dataset[i+1])
        temp.append(dataset[i+2])
        dataset_arranged.append(temp)
        
    return dataset_arranged
# print([[x] for x,i in enumerate(a) if ])

In [39]:
path_datasets =  ['integrated_datasets/apc_datasets/110.SemEval/114.restaurant14/Restaurants_Train.xml.seg',  
                 'integrated_datasets/apc_datasets/110.SemEval/115.restaurant15/restaurant_train.raw',
                 'integrated_datasets/apc_datasets/110.SemEval/116.restaurant16/restaurant_train.raw',
                 'integrated_datasets/apc_datasets/119.Yelp/yelp.train.txt'
                 ]
#                  'integrated_datasets/apc_datasets/109.MAMS/train.xml.dat']
df_integreted_dataset = pd.DataFrame()

for path in path_datasets:
    # Reading the data inside the xml
    with open(path, 'r') as f:
        data = f.read()

    # Passing the stored data inside
    # the beautifulsoup parser
    Bs_dataset = BeautifulSoup(data, "html")
    
    list_dataset = dataset_list(Bs_dataset)
    
    df_dataset = pd.DataFrame(list_dataset)
    df_integreted_dataset = pd.concat([df_integreted_dataset, df_dataset], ignore_index=True)
    
df_integreted_dataset.columns = ['text', 'aspect', 'polarity']
df_integreted_dataset

,text,aspect,polarity
0,But the $T$ was so horrible to us .,staff,Negative
1,"To be completely fair , the only redeeming fac...",food,Positive
2,"The $T$ is uniformly exceptional , with a very...",food,Positive
3,"The food is uniformly exceptional , with a ver...",kitchen,Positive
4,"The food is uniformly exceptional , with a ver...",menu,Neutral
...,...,...,...
7358,Phenomenal German cuisine! Best German $T$ in ...,food,Positive
7359,Phenomenal German cuisine! Best German food in...,potato salad,Positive
7360,"Phenomenal, is the only word I can use to desc...",food,Positive
7361,"Phenomenal, is the only word I can use to desc...",culinary,Positive


In [40]:
# there is an error in the data whgere aspects is too long
# let's first split the aspect into words to make sure we didn't delete aspects that have long words and then take the number of words as the length
df_integreted_dataset['aspects_len'] = df_integreted_dataset['aspect'].apply(lambda x: len(x.split()))
condition_above3 = df_integreted_dataset['aspects_len'] > 3
print('number of aspects with more than 3 words:',condition_above3.sum())
condition_above4 = df_integreted_dataset['aspects_len'] > 4
print('number of aspects with more than 4 words:',condition_above4.sum())
condition_above5 = df_integreted_dataset['aspects_len'] > 5
print('number of aspects with more than 5 words:',condition_above5.sum())
condition_above6 = df_integreted_dataset['aspects_len'] > 6
print('number of aspects with more than 6 words:',(condition_above6.sum()))
df_integreted_dataset = df_integreted_dataset.drop('aspects_len', axis=1)

number of aspects with more than 3 words: 248
number of aspects with more than 4 words: 117
number of aspects with more than 5 words: 47
number of aspects with more than 6 words: 29


In [41]:
# dropping aspects with more than 5 words
print('length before dropping:', df_integreted_dataset.shape[0])
df_integreted_dataset = df_integreted_dataset.drop(df_integreted_dataset[condition_above5].index, axis=0)
print('length after dropping:', df_integreted_dataset.shape[0])

length before dropping: 7363
length after dropping: 7316


In [42]:
print('duplicates in the whole dataframe :', df_integreted_dataset.duplicated().sum())
print('duplicates in the texts column :', df_integreted_dataset['text'].duplicated().sum())
print('duplicates in texts, and aspects columns :', df_integreted_dataset.loc[:, ['text', 'aspect']].duplicated().sum())

duplicates in the whole dataframe : 1209
duplicates in the texts column : 1283
duplicates in texts, and aspects columns : 1214


In [43]:
# get rid of duplicates from texts, and aspects columns
df_integreted_dataset = df_integreted_dataset.drop_duplicates(subset=['text', 'aspect'],  ignore_index=True)

print('duplicates in the whole dataframe :', df_integreted_dataset.duplicated().sum())
print('duplicates in the texts column :', df_integreted_dataset['text'].duplicated().sum())
print('duplicates in texts, and aspects columns :', df_integreted_dataset.loc[:, ['text', 'aspect']].duplicated().sum())
print('sample of the duplicates in the texts column :')
display(df_integreted_dataset[df_integreted_dataset['text'].duplicated()].head(5))

duplicates in the whole dataframe : 0
duplicates in the texts column : 69
duplicates in texts, and aspects columns : 0
sample of the duplicates in the texts column :


,text,aspect,polarity
433,Try the $T$ !,cheesecake,Positive
1474,Try the $T$ .,sea bass,Positive
1843,Good $T$ .,drink,Positive
2837,Try $T$ .,sushimi cucumber roll,Positive
3081,The $T$ was attentive .,waiter,Positive


In [44]:
print('final dataset:')
display(df_integreted_dataset)

final dataset:


,text,aspect,polarity
0,But the $T$ was so horrible to us .,staff,Negative
1,"To be completely fair , the only redeeming fac...",food,Positive
2,"The $T$ is uniformly exceptional , with a very...",food,Positive
3,"The food is uniformly exceptional , with a ver...",kitchen,Positive
4,"The food is uniformly exceptional , with a ver...",menu,Neutral
...,...,...,...
6097,Phenomenal German cuisine! Best German $T$ in ...,food,Positive
6098,Phenomenal German cuisine! Best German food in...,potato salad,Positive
6099,"Phenomenal, is the only word I can use to desc...",food,Positive
6100,"Phenomenal, is the only word I can use to desc...",culinary,Positive


In [45]:
columns_combine = df_integreted_dataset['text'] + '\n' + df_integreted_dataset['aspect'] + '\n' + df_integreted_dataset['polarity']
list_columns_combined = columns_combine.to_list()
list_columns_combined

['But the $T$ was so horrible to us .\nstaff\nNegative',
 "To be completely fair , the only redeeming factor was the $T$ , which was above average , but could n't make up for all the other deficiencies of Teodora .\nfood\nPositive",
 "The $T$ is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .\nfood\nPositive",
 "The food is uniformly exceptional , with a very capable $T$ which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .\nkitchen\nPositive",
 "The food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the $T$ or not .\nmenu\nNeutral",
 "Not only was the $T$ outstanding , but the little ` perks ' were great .\nfood\nPositive",
 "Not only was the food outstanding , but the little ' $T$ ' were great .\nperks\nPositive",
 'Our agreed favorite is the $T$ -LRB- usually the 

In [46]:
with open('dataset\custome_dataset_train.txt', 'a') as f:
    f.write('\n'.join(list_columns_combined))

In [47]:
# converting apc to atepc, make sure to rename the file and add .apc at the end
from pyabsa.utils.file_utils import convert_apc_set_to_atepc_set
convert_apc_set_to_atepc_set('dataset\custome_dataset_train.txt.apc')

C:\Users\meshari\anaconda3\envs\Graduation-Project\lib\site-packages\pyabsa\utils\pyabsa_utils.py:170: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)  # 设置主线程结束子线程立刻结束


Remote ABSADataset version: None Local ABSADatasets version: None
Unknown remote version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets
Version 1.10.4 of pyabsa is outdated. Version 1.14.0 was released 1 day ago.
check release notes at https://github.com/yangheng95/PyABSA/blob/release/release-note.json
To ensure your conversion is successful, make sure the dataset name contain "apc" and "dataset" string 
Find datasets files at dataset\custome_dataset_train.txt.apc:
dataset\custome_dataset_train.txt.apc
converting: dataset\custome_dataset_train.txt.apc
finished


### now let's do the same for the testing data

In [48]:
import pandas as pd
from bs4 import BeautifulSoup

path_datasets =  ['integrated_datasets/apc_datasets/110.SemEval/114.restaurant14/Restaurants_Test_Gold.xml.seg',  
                 'integrated_datasets/apc_datasets/110.SemEval/115.restaurant15/restaurant_test.raw',
                 'integrated_datasets/apc_datasets/110.SemEval/116.restaurant16/restaurant_test.raw',
                 'integrated_datasets/apc_datasets/119.Yelp/yelp.test.txt'
                 ]
#                  'integrated_datasets/apc_datasets/109.MAMS/test.xml.dat']
df_integreted_dataset = pd.DataFrame()
i = 1
for path in path_datasets:
    # Reading the data inside the xml
    with open(path, 'r') as f:
        data = f.read()

    # Passing the stored data inside
    # the beautifulsoup parser
    Bs_dataset = BeautifulSoup(data, "html")
    
    list_dataset = dataset_list(Bs_dataset)
    
    df_dataset = pd.DataFrame(list_dataset)
    df_integreted_dataset = pd.concat([df_integreted_dataset, df_dataset], ignore_index=True)
    
df_integreted_dataset.columns = ['text', 'aspect', 'polarity']
df_integreted_dataset

,text,aspect,polarity
0,The $T$ is top notch as well .,bread,Positive
1,I have to say they have one of the fastest $T$...,delivery times,Positive
2,$T$ is always fresh and hot - ready to eat !,Food,Positive
3,Did I mention that the $T$ is OUTSTANDING ?,coffee,Positive
4,"Certainly not the best sushi in New York , how...",place,Positive
...,...,...,...
2515,I can' t speak for much here because I don' t ...,it,Positive
2516,$T$ we had was phenomenal. I had the scallops ...,Everything,Positive
2517,Everything we had was phenomenal. I had the sc...,They,Positive
2518,Everything we had was phenomenal. I had the sc...,margarita,Positive


In [49]:
# there is an error in the data whgere aspects is too long
# let's first split the aspect into words to make sure we didn't delete aspects that have long words and then take the number of words as the length
df_integreted_dataset['aspects_len'] = df_integreted_dataset['aspect'].apply(lambda x: len(x.split()))
condition_above3 = df_integreted_dataset['aspects_len'] > 3
print('number of aspects with more than 3 words:',condition_above3.sum())
condition_above4 = df_integreted_dataset['aspects_len'] > 4
print('number of aspects with more than 4 words:',condition_above4.sum())
condition_above5 = df_integreted_dataset['aspects_len'] > 5
print('number of aspects with more than 5 words:',condition_above5.sum())
condition_above6 = df_integreted_dataset['aspects_len'] > 6
print('number of aspects with more than 6 words:',(condition_above6.sum()))
df_integreted_dataset = df_integreted_dataset.drop('aspects_len', axis=1)

number of aspects with more than 3 words: 95
number of aspects with more than 4 words: 39
number of aspects with more than 5 words: 23
number of aspects with more than 6 words: 15


In [50]:
# dropping aspects with more than 5 words
print('length before dropping:', df_integreted_dataset.shape[0])
df_integreted_dataset = df_integreted_dataset.drop(df_integreted_dataset[condition_above5].index, axis=0)
print('length after dropping:', df_integreted_dataset.shape[0])

length before dropping: 2520
length after dropping: 2497


In [51]:
print('duplicates in the whole dataframe :', df_integreted_dataset.duplicated().sum())
print('duplicates in the texts column :', df_integreted_dataset['text'].duplicated().sum())
print('duplicates in texts, and aspects columns :', df_integreted_dataset.loc[:, ['text', 'aspect']].duplicated().sum())

duplicates in the whole dataframe : 3
duplicates in the texts column : 22
duplicates in texts, and aspects columns : 5


In [52]:
# get rid of duplicates from texts, and aspects columns
df_integreted_dataset = df_integreted_dataset.drop_duplicates(subset=['text', 'aspect'],  ignore_index=True)
print('duplicates in the whole dataframe :', df_integreted_dataset.duplicated().sum())
print('duplicates in the texts column :', df_integreted_dataset['text'].duplicated().sum())
print('duplicates in texts, and aspects columns :', df_integreted_dataset.loc[:, ['text', 'aspect']].duplicated().sum())
print('sample of the duplicates in the texts column :')
display(df_integreted_dataset[df_integreted_dataset['text'].duplicated()].head(5))

duplicates in the whole dataframe : 0
duplicates in the texts column : 17
duplicates in texts, and aspects columns : 0
sample of the duplicates in the texts column :


,text,aspect,polarity
1206,great $T$ !,indian food,Positive
1342,great $T$ !,food,Positive
1378,great $T$,food,Positive
1433,great $T$,indian food,Positive
1500,great $T$,shabu shabu,Positive


In [53]:
print('final dataset:')
display(df_integreted_dataset)

final dataset:


,text,aspect,polarity
0,The $T$ is top notch as well .,bread,Positive
1,I have to say they have one of the fastest $T$...,delivery times,Positive
2,$T$ is always fresh and hot - ready to eat !,Food,Positive
3,Did I mention that the $T$ is OUTSTANDING ?,coffee,Positive
4,"Certainly not the best sushi in New York , how...",place,Positive
...,...,...,...
2487,I can' t speak for much here because I don' t ...,it,Positive
2488,$T$ we had was phenomenal. I had the scallops ...,Everything,Positive
2489,Everything we had was phenomenal. I had the sc...,They,Positive
2490,Everything we had was phenomenal. I had the sc...,margarita,Positive


In [54]:
columns_combine = df_integreted_dataset['text'] + '\n' + df_integreted_dataset['aspect'] + '\n' + df_integreted_dataset['polarity']
list_columns_combined = columns_combine.to_list()
list_columns_combined

['The $T$ is top notch as well .\nbread\nPositive',
 'I have to say they have one of the fastest $T$ in the city .\ndelivery times\nPositive',
 '$T$ is always fresh and hot - ready to eat !\nFood\nPositive',
 'Did I mention that the $T$ is OUTSTANDING ?\ncoffee\nPositive',
 'Certainly not the best sushi in New York , however , it is always fresh , and the $T$ is very clean , sterile .\nplace\nPositive',
 'I trust the $T$ at Go Sushi , it never disappoints .\npeople\nPositive',
 'Straight-forward , no surprises , very decent $T$ .\nJapanese food\nPositive',
 'BEST spicy tuna roll , great $T$ .\nasian salad\nPositive',
 'BEST $T$ , great asian salad .\nspicy tuna roll\nPositive',
 'Try the $T$ -LRB- not on menu -RRB- .\nrose roll\nPositive',
 'Try the rose roll -LRB- not on $T$ -RRB- .\nmenu\nNeutral',
 'I love the $T$ , esp lychee martini , and the food is also VERY good .\ndrinks\nPositive',
 'I love the drinks , esp $T$ , and the food is also VERY good .\nlychee martini\nPositive',
 '

In [56]:
with open('dataset\custome_dataset_test.txt', 'a') as f:
    f.write('\n'.join(list_columns_combined))

In [57]:
# converting apc to atepc, make sure to rename the file and add .apc at the end
from pyabsa.utils.file_utils import convert_apc_set_to_atepc_set
convert_apc_set_to_atepc_set('dataset\custome_dataset_test.txt.apc')

To ensure your conversion is successful, make sure the dataset name contain "apc" and "dataset" string 
Find datasets files at dataset\custome_dataset_test.txt.apc:
dataset\custome_dataset_test.txt.apc
converting: dataset\custome_dataset_test.txt.apc
finished
